### Install libraries

In [ ]:
!pip install pymupdf sentence-transformers numpy chromadb faiss-cpu qdrant-client weaviate-client pinecone-client

### Loading + Chunking Docs

In [ ]:
import fitz

In [ ]:
def load_pdf(path):
  doc = fitz.open(path)
  text = ""
  for page in doc:
    text += page.get_text()
  return text

In [ ]:
def chunk_text(text, chunk_size = 500, overlap = 100):
  chunks = []
  start = 0
  while start < len(text):
    chunks.append(text[start:start + chunk_size])
    start += chunk_size - overlap
  return chunks

In [ ]:
pdf_text = load_pdf("/content/Attention_Is_All_You_Need.pdf")

In [ ]:
chunks = chunk_text(pdf_text)

In [ ]:
print(f"Total chunks: {len(chunks)}")

### Create Embeddings

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
embeddings = model.encode(chunks).astype("float32")

In [ ]:
query = "What is multi head attention"
query_embedding = model.encode([query]).astype("float32")

## ChromaDB

In [ ]:
import chromadb
from chromadb.config import Settings

In [ ]:
client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = client.get_or_create_collection(name="pdf")

In [ ]:
collection.add(
    documents=chunks,
    embeddings=embeddings.tolist(),
    ids=[str(i) for i in range(len(chunks))]
)

In [ ]:
res = collection.query(query_texts=[query], n_results=3)
print("ChromaDB:", res["documents"][0])

## FAISS

In [ ]:
import faiss

In [ ]:
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

In [ ]:
D, I = index.search(query_embedding, k = 3)
print("FAISS: ", [chunks[i] for i in I[0]])

## Qdrant

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

In [ ]:
client = QdrantClient(":memory:")

In [ ]:
collection_name = "pdf"
vector_size = embeddings.shape[1]

In [ ]:
if client.collection_exists(collection_name):
  client.delete_collection(collection_name)

In [ ]:
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=vector_size,
        distance=Distance.COSINE
      )
)

In [ ]:
client.upsert(
    collection_name = "pdf",
    points = [
        PointStruct(id = i, vector = embeddings[i], payload = {"text": chunks[i]}) for i in range(len(chunks))
    ]
)

In [ ]:
res = client.query_points(
    collection_name = "pdf",
    query = query_embedding[0].tolist(),
    limit = 3,
    with_payload = True
)

In [ ]:
points = res.points

In [ ]:
print("Qdrant:", [p.payload["text"] for p in points])

## Weaviate

In [ ]:
import weaviate
from weaviate.connect import ConnectionParams
from weaviate.classes.config import Configure, Property, DataType
from weaviate.classes.init import Auth

In [ ]:
WEAVIATE_URL="your-weaviate-url"
WEAVIATE_API_KEY="your-weaviate-url-key"

In [ ]:
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)

In [ ]:
print(client.is_ready())

In [ ]:
client.connect()

In [ ]:
COLLECTION = "PdfChunk"

In [ ]:
if client.collections.exists(COLLECTION):
  client.collections.delete(COLLECTION)

In [ ]:
client.collections.create(
    name=COLLECTION,
    properties=[
        Property(name="text", data_type=DataType.TEXT),
    ],
    vectorizer_config = Configure.Vectorizer.none()
)

In [ ]:
collection = client.collections.get(COLLECTION)

for i, emb in enumerate(embeddings):
  collection.data.insert(
      properties={"text":chunks[i]},
      vector=emb.tolist(),
  )

In [ ]:
res = collection.query.near_vector(
    near_vector=query_embedding[0].tolist(),
    limit=3
)

In [ ]:
print("Weaviate:", [obj.properties["text"] for obj in res.objects])

## Pinecone

In [ ]:
from pinecone import Pinecone

In [ ]:
PINECONE_API_KEY = "your-pinecone-api-key"

In [ ]:
pc = Pinecone(api_key = PINECONE_API_KEY)
index = pc.Index("pdf-index")

In [ ]:
index.upsert(
    vectors = [(str(i), embeddings[i].tolist()) for i in range(len(chunks))]
)

In [ ]:
res = index.query(
    vector = query_embedding[0].tolist(),
    top_k = 3,
    include_metadata =False
)

In [ ]:
print("Pinecone:", [m["id"] for m in res["matches"]])